# **Average Distance Check**

In [ ]:
from ase.io import read
import numpy as np
from tkinter.filedialog import askopenfilename

# Set distance between atoms (unit: Å)
min_cutoff = 1.7
max_cutoff = 2.3

# select structure file (POSCAR or CONTCAR)
file_path = askopenfilename(title="Select structure file",
                            filetypes=[("Structure files", "*.cif *.vasp *POSCAR* *CONTCAR*")])

atoms = read(file_path)

# Extract atoms index
ni_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Ni']
fe_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Fe']
o_indices  = [i for i, atom in enumerate(atoms) if atom.symbol == 'O']

# Calculate average distance (min_cutoff ≤ distance ≤ max_cutoff)
distances_1 = []
Ni_each = []
Fe_each = []
for i in ni_indices:
    ni_temp = []
    for j in o_indices:
        d = atoms.get_distance(i, j, mic=True)
        if min_cutoff <= d <= max_cutoff:
            distances_1.append(d)
            ni_temp.append(d)
    Ni_each.append(np.mean(ni_temp))


distances_2 = []
for i in fe_indices:
    fe_temp = []
    for j in o_indices:
        d = atoms.get_distance(i, j, mic=True)
        if min_cutoff <= d <= max_cutoff:
            distances_2.append(d)
            fe_temp.append(d)
    Fe_each.append(np.mean(fe_temp))

# Print distances
if distances_1:
    mean_distance_1 = np.mean(distances_1)
    mean_distance_2 = np.mean(distances_2)
    print(f"Average Ni–O distance within {min_cutoff}-{max_cutoff} Å: {mean_distance_1:.2f} Å")
    print(f"Average Fe–O distance within {min_cutoff}-{max_cutoff} Å: {mean_distance_2:.2f} Å")
    print(f"Number of valid Ni–O pairs: {len(distances_1)}")
    print(f"Number of valid Fe–O pairs: {len(distances_2)}")
else:
    print(f"No Ni–O pairs found within the {min_cutoff}-{max_cutoff} Å range.")

for i in Ni_each :
    print(f"Ni : {i:.2f}")

for i in Fe_each :
    print(f"Fe : {i:.2f}")

In [ ]:
print(distances_1)

# **Show Final Energies of OSZICAR**


In [ ]:
import os
import pandas as pd
from tkinter import filedialog, Tk


def find_last_F_line(oszicar_path):
    try:
        with open(oszicar_path, 'r') as f:
            lines_with_F = [line.strip() for line in f if 'F' in line]
        return lines_with_F[-1] if lines_with_F else None
    except Exception as e:
        return f"Error reading file: {e}"

def search_oszicar_in_folders(root_dir):
    data = []
    for dirpath, _, filenames in os.walk(root_dir):
        if 'OSZICAR' in filenames:
            oszicar_path = os.path.join(dirpath, 'OSZICAR')
            last_line = find_last_F_line(oszicar_path)
            if last_line:
                rel_path = os.path.relpath(dirpath, root_dir)
                data.append((rel_path, last_line))
    df = pd.DataFrame(data, columns=["Folder", "Last F Line"])

    split_cols = df["Last F Line"].str.split(expand=True)
    split_cols.columns = [f"Field_{i}" for i in range(split_cols.shape[1])]

    selected = pd.concat([df["Folder"], split_cols[["Field_3", "Field_4"]]], axis=1)
    return selected



Tk().withdraw()
root_folder = filedialog.askdirectory(title="Select the directory for OSZICAR")
if not root_folder:
    print("No directory selected")
    exit()

df = search_oszicar_in_folders(root_folder)
if df.empty:
    print("No ionic step is detected in OSZICAR file")
else:
    display(df)  


In [ ]:
home_dir = os.path.join(os.path.expanduser("~"), "Desktop")
save_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                        filetypes=[("CSV files", "*.csv")],
                                        title="Save OSZICAR summary as CSV")

df.to_csv(save_path, index=False, encoding='utf-8-sig')

# **Extract middle structure from OUTCAR file**

In [ ]:
import os
import glob
from ase.io import read, write
from tkinter import Tk
from tkinter.filedialog import askopenfilename


def read_poscar_header(poscar_path):
    with open(poscar_path, 'r') as f:
        lines = f.readlines()
    offset = 7
    if lines[offset].strip().lower().startswith('s'):
        offset += 1
    offset += 1
    return lines[:offset]


def parse_outcar_coordinates(outcar_path, natoms):
    traj = []
    with open(outcar_path, 'r') as f:
        lines = f.readlines()
    i = 0
    while i < len(lines):
        if lines[i].strip().startswith("POSITION"):
            i += 2
            coords = []
            for _ in range(natoms):
                parts = lines[i].split()
                coords.append(parts[:3])
                i += 1
            traj.append(coords)
        else:
            i += 1
    return traj


Tk().withdraw()
print("Select POSCAR file:")
poscar_file = askopenfilename(title="Select POSCAR",
                              filetypes=[("POSCAR","POSCAR*"),("All files","*.*")])
if not poscar_file:
    print("No POSCAR selected. Exiting."); exit()

print("Select OUTCAR file:")
outcar_file = askopenfilename(title="Select OUTCAR",
                              filetypes=[("OUTCAR","OUTCAR*"),("All files","*.*")])
if not outcar_file:
    print("No OUTCAR selected. Exiting."); exit()



header = read_poscar_header(poscar_file)


atoms_ref = read(poscar_file)
natoms = len(atoms_ref)


coord_steps = parse_outcar_coordinates(outcar_file, natoms)
if not coord_steps:
    print("No POSITION blocks found in OUTCAR. Exiting."); exit()


poscar_dir = os.path.dirname(poscar_file)
for idx, coords in enumerate(coord_steps, 1):
    fname = os.path.join(poscar_dir, f"POSCAR_{idx:04d}")
    with open(fname, 'w') as f:
        f.writelines(header)
        for c in coords:
            f.write("  " + "  ".join(c) + "\n")
    print(f"Wrote {fname}")


frames = []
for coords in coord_steps:
    frame = atoms_ref.copy()
    frame.set_positions([[float(x) for x in c] for c in coords])
    frames.append(frame)


out_traj = os.path.join(poscar_dir, "merged.traj")
write(out_traj, frames, format="traj")


pattern = os.path.join(poscar_dir, "POSCAR_*")
for path in glob.glob(pattern):
    try:
        os.remove(path)
        print(f"Deleted {path}")
    except OSError as e:
        print(f"Failed to delete {path}: {e}")


print(f"Saved merged trajectory to {out_traj} ({len(frames)} frames)")


## **Visualization**

In [ ]:
from ase.io import read
from ase.gui.gui import GUI
from tkinter.filedialog import askopenfilename

def open_file():
    fname = askopenfilename(
        title="Select structure file",
        filetypes=[("All supported files", "*.cif *.xyz *.vasp *.pdb *.traj"),
                   ("All files", "*.*")]
    )
    if not fname:
        print("No file selected.")
        return

    try:
        atoms = read(fname, index=':')
        if not atoms:
            print("No structure(s) found in file.")
            return
    except Exception as e:
        print(f"Failed to read file: {e}")
        return

    gui = GUI(images=atoms)
    gui.run()

open_file()

# 🔬 **Visualization Tool using ASE**

**Method:**  
Use a File Dialog and Select Structure File or Trajectory File

**Supported Features:**

* Supported file formats: `.cif`, `.xyz`, `VASP`, `.pdb`, `.traj`
* Opens in a separate interactive window
* Does not block the Jupyter Notebook cell
* Returns the loaded structure as an `ASE Atoms` object

**⚠️ Notes:**

* The GUI window must be closed before proceeding to the next cell
* X11 forwarding may be required on Linux/WSL environments

In [4]:
import os
from ase.io import read
from ase.gui.gui import GUI
from tkinter.filedialog import askopenfilename
import threading
import warnings

# Ignore GUI related warning messages in Jupyter condition
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

def open_file_dialog():
    """Select structure file through filedialog box"""
    fname = askopenfilename(
        title="Select structure file",
        filetypes=[
            ("VASP files", "*.vasp *.poscar *.contcar *POSCAR* *CONTCAR*"),
            ("CIF files", "*.cif"), 
            ("XYZ files", "*.xyz"),
            ("PDB files", "*.pdb"),
            ("Trajectory files", "*.traj"),
            ("All supported files", "*.cif *.xyz *.vasp *.poscar *.contcar *.pdb *.traj"),
            ("All files", "*.*")
        ]
    )
    return fname

def load_structure(filename):
    """load structure file and extract informations"""
    if not filename:
        print("❌ No file selected.")
        return None
    
    if not os.path.exists(filename):
        print(f"❌ Error: File '{filename}' does not exist.")
        return None
    
    try:
        print(f"📁 Reading file: {os.path.basename(filename)}")
        try:
            atoms = read(filename, index=':')
        except Exception:
            # if reading fails, try to read as a single structure
            atoms = read(filename)
        
        if not atoms:
            print("❌ No structure(s) found in file.")
            return None
            
        return atoms
        
    except Exception as e:
        print(f"❌ Failed to read file '{os.path.basename(filename)}': {e}")
        return None

def open_gui_threaded(atoms):
    """Run ASE GUI in a separate thread to avoid blocking Jupyter"""
    try:
        print("🖥️  Opening ASE GUI...")
        print("(GUI will open in a separate window)")
        gui = GUI(images=atoms)
        gui.run()
        print("✅ ASE GUI closed.")
    except Exception as e:
        print(f"❌ Failed to open GUI: {e}")

def open_file():
    """Main function to open file dialog and load structure"""
    # Select file using dialog
    filename = open_file_dialog()
    
    # Load structure from selected file
    atoms = load_structure(filename)
    if atoms is None:
        return None
    
    # Run GUI in a separate thread
    gui_thread = threading.Thread(target=open_gui_threaded, args=(atoms,))
    gui_thread.daemon = True
    gui_thread.start()
    
    return atoms


# Show usage instructions
open_file()

📁 Reading file: CONTCAR
🖥️  Opening ASE GUI...
(GUI will open in a separate window)


[Atoms(symbols='CHBi48O58', pbc=True, cell=[[14.8442, 0.0, 0.0], [3.146696083791301, 13.53913969778924, 0.0], [0.0, 0.0, 28.8866]], constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103]))]

✅ ASE GUI closed.
